In [1]:
%pip install opendatasets openai unstructured[pdf] gradio langchain-openai aperturedb pandas langchain-community arxiv --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [22]:
import os
import json
import arxiv
import requests
import pandas as pd
import opendatasets as od
from langchain_core.documents import Document
from unstructured.partition.auto import partition
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredPDFLoader

In [3]:
dataset = 'https://www.kaggle.com/datasets/Cornell-University/arxiv'
od.download(dataset)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: syedhamza17
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/Cornell-University/arxiv


100%|██████████| 1.36G/1.36G [01:02<00:00, 23.3MB/s]


In [4]:
def fetch_paper_details(arxiv_id):
    paper = next(arxiv.Client().results(arxiv.Search(id_list=[arxiv_id])))
    paper.download_pdf( filename=f"{arxiv_id}.pdf")
    return partition(f"{arxiv_id}.pdf")

In [32]:
papers = []
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=5000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

sample = 20 # Arxiv has over 1.7M articles, using 20 for our application

# Open the JSON file and process entries
with open("arxiv/arxiv-metadata-oai-snapshot.json", "r") as file:
    for _ in range(sample):
        line = file.readline()
        data = json.loads(line)

        # Extract relevant fields
        arxiv_id = data.get("id", "")

        # Add paper details by downloading and parsing the paper
        paper_details = "".join(
            text if isinstance((text := element.text), str)
            else "".join(str(part) for part in text) if isinstance(text, (list, tuple))
            else str(text)
            for element in fetch_paper_details(arxiv_id)
        )
        print(type(paper_details))
        # Use LangChain's splitter to divide paper details into chunks
        chunks = text_splitter.create_documents([paper_details])
        print(len(chunks))
        # Create a Document for each chunk
        for idx, chunk in enumerate(chunks):
            print(chunk,type(chunk))
            document_id = f"{arxiv_id}_{idx + 1}"  # Unique ID for each chunk
            document = Document(
                page_content=chunk.page_content,
                id=document_id,
                metadata={
                    'title': data.get("title",""),
                    'authors': data.get("authors", ""),
                    'submitter': data.get("submitter", ""),
                    'abstract': data.get("abstract", ""),
                    'paper_content': chunk.page_content
                }
            )
            papers.append(document)

print("Processing complete. Papers saved to processed_papers.json.")

<class 'str'>
13
ts the angle betweenP µ3 =P µ4 =Q 2 { Q 2 {0, sin θ∗ cos ϕ∗, sin θ∗ sin ϕ∗, cos θ∗} 0,sin θ∗ cos ϕ∗,sin θ∗ sin ϕ∗,−−−,cos θ∗},where θ∗ and ϕ∗ are the photon's polar and azimuthal angles.' <class 'langchain_core.documents.base.Document'>
at- γγ . The Sudakov tering a¯a(cid:3) γγ , and a' <class 'langchain_core.documents.base.Document'>
es with oppositegenerates the term proportional to Σ′g(θ∗, ϕ∗)∝cos 2ϕ∗ in the small-QT' <class 'langchain_core.documents.base.Document'>
ulation at small QT , for the nominal Eiso T .photon pairs with relatively small mass Q.the leading part Eq. (20) of' <class 'langchain_core.documents.base.Document'>
ontributions P (Q, QT , y, Ω∗) and A(Q, QT , y, Ω∗). We use two-loop expressions for the running' <class 'langchain_core.documents.base.Document'>
tion P (dashed, same as in (a)) at large QT .→between the P and A distributions' <class 'langchain_core.documents.base.Document'>
h' <class 'langchain_core.documents.base.Document'>
tion at small

In [34]:
papers

[Document(id='0704.0001_1', metadata={'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies', 'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan", 'submitter': 'Pavel Nadolsky', 'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are shown for\ndistributions of diphoton pairs produced at the energy of the Large Hadron\nColli

In [8]:
!adb config create --active --from-json

Enter JSON string: 


In [35]:
from langchain_community.vectorstores import ApertureDB

embeddings = OpenAIEmbeddings(api_key  = "<API-KEY>")
vector_db = ApertureDB.from_documents(papers, embeddings)

In [50]:
# Create prompt

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

{context}


Question: {input}""")
llm = ChatOpenAI(
    api_key  = "<API-KEY>",
    model="gpt-4o",
    temperature=0,
)
# Create a chain that passes documents to an LLM
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)


# Treat the vectorstore as a document retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 1})


# Create a RAG chain that connects the retriever to the LLM
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [58]:
def ask_question(question,history):
    response = retrieval_chain.invoke({"input": question})
    return response["answer"]


In [52]:
ask_question("What is this paper about: Calulation of prompt diphoton prodution ross setions at Tevatron and LHC energies")

{'input': 'What is this paper about: Calulation of prompt diphoton prodution ross setions at Tevatron and LHC energies',
 'context': [Document(id='0704.0001_6', metadata={'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are shown for\ndistributions of diphoton pairs produced at the energy of the Large Hadron\nCollider (LHC). Distributions of the diphoton pairs from the decay of a H

In [59]:
import gradio as gr

gr.ChatInterface(ask_question, type="messages").launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f49f090dce48b5239d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://f49f090dce48b5239d.gradio.live
